In [1]:
import tensorflow as tf

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import sklearn.metrics
import os

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Training on C1 and C2 and Testing on C3

In [3]:
NS_case1 = pd.read_excel("../NS_EOS_Case1.xlsx")
NS_case2 = pd.read_excel("../NS_EOS_Case2.xlsx")
NS_case3 = pd.read_excel("../NS_EOS_Case3.xlsx")
NS = pd.concat([NS_case1,NS_case2],ignore_index=True)
X = NS.iloc[:, 0:7].values
y = NS.iloc[:, [7,9,11]].values
X_test = NS_case3.iloc[:, 0:7].values
y_test = NS_case3.iloc[:, [7,9,11]].values
#Standardizing the data
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X = sc_X.fit_transform(X)
X_test = sc_X.transform(X_test)
sc_y = StandardScaler()
y = sc_y.fit_transform(y)

## Define the 2 HL model definition

In [17]:
def HL_2_model(i,j,result_new):
    #create model
    model = Sequential()  #creating sequential model; Dense specifies fully connected layer
    no_of_hidden_layers = 2
    node_in_each_layer = [i,j]
    model.add(Dense(node_in_each_layer[0],input_dim=7,activation='relu',kernel_initializer='normal')) #first hidden layer
    model.add(Dense(node_in_each_layer[1],activation='relu',kernel_initializer='normal')) #second hidden layer
    model.add(Dense(3,kernel_initializer='normal'))
    #compile model
    model.compile(loss='mean_squared_error',optimizer='adam',metrics=['mean_squared_error'])
    #Train model
    train = model.fit(X,y,epochs=50,batch_size=16)  #size of dataset between 32-512
    #check performance
    y_pred = model.predict(X_test)

    y_pred = sc_y.inverse_transform(y_pred)
    #RMS value
    rms = sklearn.metrics.mean_squared_error(y_test, y_pred)**0.5
    rms_mass = sklearn.metrics.mean_squared_error(y_test[:,0], y_pred[:,0])**0.5
    rms_rad = sklearn.metrics.mean_squared_error(y_test[:,1], y_pred[:,1])**0.5
    rms_lam = sklearn.metrics.mean_squared_error(y_test[:,2], y_pred[:,2])**0.5

    #R2 value
    r2 = sklearn.metrics.r2_score(y_test,y_pred)
    
    #Selection Criterion
    flag=0
    ext_dir = ''
    if rms<4 and r2>0.99:     
        flag = 1
        ext_dir = 'Good_models'
    else:
        flag=2
        ext_dir = 'Bad_models'
    result_new = result_new.append({'Model':'ANN(2HL-'+str(i)+','+str(j)+' Nodes)', 'RMS':rms,'RMS_NS_Mass':rms_mass,'RMS_NS_Rad':rms_rad,
                            'RMS_Lam':rms_lam, 'R2':r2, 'Flag':flag}, ignore_index=True)
    #create and open folder   
    dire = os.path.join('2HL',ext_dir,'HL_nodes-'+str(i)+'_'+str(j))
    if not os.path.exists(dire):
        os.mkdir(dire)
    plt.plot(train.history['loss'])
    plt.title("Training loss curve (50 epochs)")
    direc = os.path.join(dire,'Training loss curve (50 epochs)')
    plt.savefig(direc)
    plt.close()
    #Plotting
    titles = ['NS_Mass','NS_Rad','Lam']
    for i in range(3):
        rms = 'RMSE:'+str((np.sqrt(sklearn.metrics.mean_squared_error(y_test[:,i], y_pred[:,i]))))
        fig, ax = plt.subplots()
        ax.scatter(y_test[:,i], y_pred[:,i])
        ax.plot([y_test[:,i].min(), y_test[:,i].max()], [y_test[:,i].min(), y_test[:,i].max()], 'k--', lw=4)
        ax.set_xlabel('Actual')
        ax.set_ylabel('Predicted')
        plt.title(titles[i]+';ANN(2HL-'+str(i)+','+str(j)+' Nodes);[Case1+Case2 on Case3] \n'+rms)
        direc = os.path.join(dire,titles[i])
        plt.savefig(direc)
        plt.close()
    return result_new
    

In [18]:
result_new = pd.DataFrame(columns=['Model','RMS','RMS_NS_Mass','RMS_NS_Rad','RMS_Lam','R2','Flag'])
for i in range(1,21):
    for j in range(1,21):
        result_new = HL_2_model(i,j,result_new)
result_new

Train on 3972 samples
Epoch 1/50
3972/3972 [==============================] - 0s 118us/sample - loss: 1.0002 - mean_squared_error: 1.0002
Epoch 2/50
3972/3972 [==============================] - 0s 61us/sample - loss: 1.0002 - mean_squared_error: 1.0002
Epoch 3/50
3972/3972 [==============================] - 0s 62us/sample - loss: 1.0003 - mean_squared_error: 1.0003
Epoch 4/50
3972/3972 [==============================] - 0s 64us/sample - loss: 1.0002 - mean_squared_error: 1.0002
Epoch 5/50
3972/3972 [==============================] - 0s 61us/sample - loss: 1.0002 - mean_squared_error: 1.0002
Epoch 6/50
3972/3972 [==============================] - 0s 62us/sample - loss: 1.0002 - mean_squared_error: 1.0002
Epoch 7/50
3972/3972 [==============================] - 0s 61us/sample - loss: 1.0002 - mean_squared_error: 1.0002
Epoch 8/50
3972/3972 [==============================] - 0s 64us/sample - loss: 1.0001 - mean_squared_error: 1.0001
Epoch 9/50
3972/3972 [==============================] - 0

3972/3972 [==============================] - 0s 61us/sample - loss: 0.3683 - mean_squared_error: 0.3683
Epoch 38/50
3972/3972 [==============================] - 0s 60us/sample - loss: 0.3683 - mean_squared_error: 0.3683
Epoch 39/50
3972/3972 [==============================] - 0s 60us/sample - loss: 0.3683 - mean_squared_error: 0.3683
Epoch 40/50
3972/3972 [==============================] - 0s 61us/sample - loss: 0.3685 - mean_squared_error: 0.3685
Epoch 41/50
3972/3972 [==============================] - 0s 61us/sample - loss: 0.3682 - mean_squared_error: 0.3682
Epoch 42/50
3972/3972 [==============================] - 0s 60us/sample - loss: 0.3682 - mean_squared_error: 0.3682
Epoch 43/50
3972/3972 [==============================] - 0s 61us/sample - loss: 0.3680 - mean_squared_error: 0.3680
Epoch 44/50
3972/3972 [==============================] - 0s 61us/sample - loss: 0.3681 - mean_squared_error: 0.3681
Epoch 45/50
3972/3972 [==============================] - 0s 61us/sample - loss: 0.36

Epoch 23/50
3972/3972 [==============================] - 0s 61us/sample - loss: 0.3771 - mean_squared_error: 0.3771
Epoch 24/50
3972/3972 [==============================] - 0s 60us/sample - loss: 0.3761 - mean_squared_error: 0.3761
Epoch 25/50
3972/3972 [==============================] - 0s 61us/sample - loss: 0.3751 - mean_squared_error: 0.3751
Epoch 26/50
3972/3972 [==============================] - 0s 62us/sample - loss: 0.3742 - mean_squared_error: 0.3742
Epoch 27/50
3972/3972 [==============================] - 0s 61us/sample - loss: 0.3737 - mean_squared_error: 0.3737
Epoch 28/50
3972/3972 [==============================] - 0s 61us/sample - loss: 0.3732 - mean_squared_error: 0.3732
Epoch 29/50
3972/3972 [==============================] - 0s 61us/sample - loss: 0.3726 - mean_squared_error: 0.3726
Epoch 30/50
3972/3972 [==============================] - 0s 61us/sample - loss: 0.3722 - mean_squared_error: 0.3722
Epoch 31/50
3972/3972 [==============================] - 0s 61us/sample 

3972/3972 [==============================] - 0s 62us/sample - loss: 0.4126 - mean_squared_error: 0.4126
Epoch 8/50
3972/3972 [==============================] - 0s 60us/sample - loss: 0.4031 - mean_squared_error: 0.4031
Epoch 9/50
3972/3972 [==============================] - 0s 60us/sample - loss: 0.3960 - mean_squared_error: 0.3960
Epoch 10/50
3972/3972 [==============================] - 0s 61us/sample - loss: 0.3905 - mean_squared_error: 0.3905
Epoch 11/50
3972/3972 [==============================] - 0s 60us/sample - loss: 0.3865 - mean_squared_error: 0.3865
Epoch 12/50
3972/3972 [==============================] - 0s 60us/sample - loss: 0.3836 - mean_squared_error: 0.3836
Epoch 13/50
3972/3972 [==============================] - 0s 61us/sample - loss: 0.3812 - mean_squared_error: 0.3812
Epoch 14/50
3972/3972 [==============================] - 0s 60us/sample - loss: 0.3793 - mean_squared_error: 0.3793
Epoch 15/50
3972/3972 [==============================] - 0s 61us/sample - loss: 0.3780

3972/3972 [==============================] - 0s 61us/sample - loss: 0.3684 - mean_squared_error: 0.3684
Epoch 43/50
3972/3972 [==============================] - 0s 60us/sample - loss: 0.3684 - mean_squared_error: 0.3684
Epoch 44/50
3972/3972 [==============================] - 0s 61us/sample - loss: 0.3682 - mean_squared_error: 0.3682
Epoch 45/50
3972/3972 [==============================] - 0s 60us/sample - loss: 0.3679 - mean_squared_error: 0.3679
Epoch 46/50
3972/3972 [==============================] - 0s 60us/sample - loss: 0.3680 - mean_squared_error: 0.3680
Epoch 47/50
3972/3972 [==============================] - 0s 60us/sample - loss: 0.3682 - mean_squared_error: 0.3682
Epoch 48/50
3972/3972 [==============================] - 0s 60us/sample - loss: 0.3681 - mean_squared_error: 0.3681
Epoch 49/50
3972/3972 [==============================] - 0s 61us/sample - loss: 0.3680 - mean_squared_error: 0.3680
Epoch 50/50
3972/3972 [==============================] - 0s 62us/sample - loss: 0.36

...

3972/3972 [==============================] - 0s 73us/sample - loss: 0.0613 - mean_squared_error: 0.0613
Epoch 34/50
3972/3972 [==============================] - 0s 70us/sample - loss: 0.0606 - mean_squared_error: 0.0606
Epoch 35/50
3972/3972 [==============================] - 0s 71us/sample - loss: 0.0599 - mean_squared_error: 0.0599s - loss: 0.0446 - mean_squared_error
Epoch 36/50
3972/3972 [==============================] - 0s 69us/sample - loss: 0.0593 - mean_squared_error: 0.0593
Epoch 37/50
3972/3972 [==============================] - 0s 70us/sample - loss: 0.0583 - mean_squared_error: 0.0583
Epoch 38/50
3972/3972 [==============================] - 0s 68us/sample - loss: 0.0584 - mean_squared_error: 0.0584
Epoch 39/50
3972/3972 [==============================] - 0s 69us/sample - loss: 0.0572 - mean_squared_error: 0.0572
Epoch 40/50
3972/3972 [==============================] - 0s 68us/sample - loss: 0.0571 - mean_squared_error: 0.0571
Epoch 41/50
3972/3972 [========================

3972/3972 [==============================] - 0s 65us/sample - loss: 0.0480 - mean_squared_error: 0.0480
Epoch 50/50
3972/3972 [==============================] - 0s 61us/sample - loss: 0.0481 - mean_squared_error: 0.0481
Train on 3972 samples
Epoch 1/50
3972/3972 [==============================] - 0s 123us/sample - loss: 0.5759 - mean_squared_error: 0.5759
Epoch 2/50
3972/3972 [==============================] - 0s 68us/sample - loss: 0.2221 - mean_squared_error: 0.2221
Epoch 3/50
3972/3972 [==============================] - 0s 64us/sample - loss: 0.1334 - mean_squared_error: 0.1334
Epoch 4/50
3972/3972 [==============================] - ETA: 0s - loss: 0.1141 - mean_squared_error: 0.11 - 0s 64us/sample - loss: 0.1122 - mean_squared_error: 0.1122
Epoch 5/50
3972/3972 [==============================] - 0s 66us/sample - loss: 0.1028 - mean_squared_error: 0.1028
Epoch 6/50
3972/3972 [==============================] - 0s 65us/sample - loss: 0.0948 - mean_squared_error: 0.0948s - loss: 0.1088

3972/3972 [==============================] - 0s 67us/sample - loss: 0.0682 - mean_squared_error: 0.0682
Epoch 19/50
3972/3972 [==============================] - 0s 66us/sample - loss: 0.0669 - mean_squared_error: 0.0669
Epoch 20/50
3972/3972 [==============================] - 0s 69us/sample - loss: 0.0659 - mean_squared_error: 0.0659
Epoch 21/50
3972/3972 [==============================] - 0s 70us/sample - loss: 0.0653 - mean_squared_error: 0.0653
Epoch 22/50
3972/3972 [==============================] - 0s 71us/sample - loss: 0.0643 - mean_squared_error: 0.0643
Epoch 23/50
3972/3972 [==============================] - 0s 70us/sample - loss: 0.0626 - mean_squared_error: 0.0626
Epoch 24/50
3972/3972 [==============================] - 0s 68us/sample - loss: 0.0615 - mean_squared_error: 0.0615
Epoch 25/50
3972/3972 [==============================] - 0s 66us/sample - loss: 0.0606 - mean_squared_error: 0.0606
Epoch 26/50
3972/3972 [==============================] - 0s 65us/sample - loss: 0.05

Epoch 4/50
3972/3972 [==============================] - 0s 65us/sample - loss: 0.0924 - mean_squared_error: 0.0924
Epoch 5/50
3972/3972 [==============================] - 0s 67us/sample - loss: 0.0850 - mean_squared_error: 0.0850
Epoch 6/50
3972/3972 [==============================] - 0s 64us/sample - loss: 0.0809 - mean_squared_error: 0.0809
Epoch 7/50
3972/3972 [==============================] - 0s 64us/sample - loss: 0.0778 - mean_squared_error: 0.0778
Epoch 8/50
3972/3972 [==============================] - 0s 65us/sample - loss: 0.0764 - mean_squared_error: 0.0764
Epoch 9/50
3972/3972 [==============================] - 0s 65us/sample - loss: 0.0749 - mean_squared_error: 0.0749
Epoch 10/50
3972/3972 [==============================] - 0s 70us/sample - loss: 0.0738 - mean_squared_error: 0.0738
Epoch 11/50
3972/3972 [==============================] - 0s 65us/sample - loss: 0.0725 - mean_squared_error: 0.0725
Epoch 12/50
3972/3972 [==============================] - 0s 65us/sample - loss

,Model,RMS,RMS_NS_Mass,RMS_NS_Rad,RMS_Lam,R2,Flag
0,"ANN(2HL-1,1, Nodes)",69.319016,0.079979,0.662370,120.062203,-0.007571,2
1,"ANN(2HL-1,2, Nodes)",69.357122,0.079983,0.661961,120.128209,-0.007561,2
2,"ANN(2HL-1,3, Nodes)",8.867608,0.057766,0.216108,15.357518,0.782765,2
3,"ANN(2HL-1,4, Nodes)",9.305709,0.057622,0.226748,16.116263,0.779479,2
4,"ANN(2HL-1,5, Nodes)",8.699095,0.057578,0.213542,15.065652,0.784958,2
5,"ANN(2HL-1,6, Nodes)",9.402346,0.058222,0.204206,16.283956,0.783117,2
6,"ANN(2HL-1,7, Nodes)",9.820988,0.058617,0.205286,17.009111,0.779784,2
7,"ANN(2HL-1,8, Nodes)",8.539570,0.057794,0.211038,14.789351,0.784646,2
8,"ANN(2HL-1,9, Nodes)",10.027883,0.058153,0.211125,17.367422,0.780497,2
9,"ANN(2HL-1,10, Nodes)",8.734886,0.057557,0.207812,15.127729,0.786887,2


In [1]:
#Segregating bad and good models
result_new_good = result_new[result_new['Flag']==1]
result_new_bad = result_new[result_new['Flag']==2]
result_new_v_good1 = result_new_good[result_new_good['R2']>0.99 ]  #Comment out later
result_new_v_good = result_new_v_good1[result_new_v_good1['RMS']<4]
result_new_v_good = result_new_v_good.reset_index()
result_new_v_good = result_new_v_good.drop(['index','Flag'],axis=1)
result_new_v_good

NameError: name 'result_new' is not defined

In [42]:
#Writing in file
direc1 = os.path.join('2HL','Bad_Model_details.xlsx')
result_new_bad.to_excel(direc1)
direc2 = os.path.join('2HL','Good_Model_details.xlsx')
result_new_good.to_excel(direc2)
direc3 = os.path.join('2HL','VGood_Model_details.xlsx')
result_new_v_good.to_excel(direc3)

In [56]:
list_of_V_Good_models = []
names = result_new_v_good['Model'].tolist()
for x in names:
    ij = x[8:-8].split(',')
    list_of_V_Good_models.append((int(ij[0]),int(ij[1])))
list_of_V_Good_models

[(12, 15),
 (13, 16),
 (13, 19),
 (14, 11),
 (14, 16),
 (14, 20),
 (15, 13),
 (15, 16),
 (15, 17),
 (15, 19),
 (16, 8),
 (16, 15),
 (16, 17),
 (16, 20),
 (17, 8),
 (17, 12),
 (17, 16),
 (17, 17),
 (17, 18),
 (17, 19),
 (18, 6),
 (18, 14),
 (18, 16),
 (18, 19),
 (19, 6),
 (19, 8),
 (19, 9),
 (19, 10),
 (19, 11),
 (19, 12),
 (19, 14),
 (19, 18),
 (19, 19),
 (19, 20),
 (20, 7),
 (20, 8),
 (20, 9),
 (20, 12),
 (20, 13),
 (20, 15),
 (20, 17),
 (20, 18),
 (20, 19),
 (20, 20)]

In [2]:
list_of_V_Good_models= [(12, 15),(13, 16),(14, 11),(15, 16),(15, 19),(17, 18),(18, 16),(19, 11),(19, 14),(19, 18),(19, 20)]